---
title: stata教程09-项目反应理论IRT的Rash模型
date: 2018-12-22 17:17:55
tags: [stata]
toc: true
mathjax: true

---

本篇文章主要演示了一下如何使用固定效应的条件逻辑回归模型来估计题目难度。

<span></span>
<!-- more -->

> 本篇文章部分来自于stata的官方文档, 英文好的建议读原文: https://www.stata.com/support/faqs/statistics/rasch-model/

### 背景知识

Rasch模型是心理测量学中二分类题目（例如，选择题只有正确和错误之分）的主要模型之一。设$y_{ij}$取值只有(0或者1)，其中$i=1,...,n$，n是被试人数，$j = 1，...，m$，m是题目的数量。 Rasch模型可以写为logit-linear模型：

$$
logit P(y_{ij}=1 | \eta_i) = \eta_i − \theta_j
$$

上面式子中$\eta_i$指的是被试i的能力值, $\theta_j$指的是题目的难度值。


此外，假设以$\eta_i$为条件，$y_i$相对于$i$是独立的（“局部独立性”）。实际上，丹麦统计学家Rash在20世纪60年代给出了该模型的公理推导，其中，除了“局部独立性”之外，主要特征是{y（+ j），y（i +）}形成了足够的统计量为{eta，theta}。

当将$\eta$和$\theta$作为参数（固定效应）处理时，人们早就知道最大似然估计在标准的逼近过程中是不一致的（n→无穷大，m是固定的）。我们每多测试一个被试，我们就会多了m个观测数据, 就会有一个额外的$\eta$参数需要估计。因此，直观地说，随着n的增加, 每个参数的观测数据以及信息量是保持不变的。

在20世纪80年代，Andersen展示了$\theta_j$的条件极大似然（CML）估计(条件就是每个被试在所有题目上的得分$y_{i+}$)是有效的并且实际上是渐近正态分布的，在正常情况下具有ml估计的所有良好属性。例如，条件似然比检验具有类似于标准似然比检验的性质。虽然已经研究了其他估计量，但CML估计量是最广泛用于固定效应情况的估算器 （Fischer和Molenaar 1995）。可以使用clogit命令拟合此模型。

心理测量学家也研究了高斯随机效应估计器，这可能只是一个小小的惊喜。从Stata 14开始，使用irt 1pl可以拟合数学上等效的模型。从Stata 13开始，Rasch模型可以使用gsem;见[SEM]例子28g。在Stata 13之前，Rasch模型可以通过随机效果面板估算器拟合，由xtlogit，re命令计算。

### 使用$\eta$作为固定效应拟合Rasch模型

考虑最可能的情况;有一个数据集，每一行代表一个被试在所有题上的得分, 每一列代表同一个题目下所有被试的得分。你可以使用clogit中的条件logit固定效应估计器获得Rasch模型的θ参数的CLM估计值（xtlogit，fe相当于键入clogit）。该命令要求所有得分保存为一个变量，而“组”变量用于识别属于同一被试的分数。这可以使用reshape命令完成。最后，可以将Rasch模型描述为具有m个协变量$x_{ijk}$，$k = 1，...，m$的“clogit模型”，因此对于所有i，如果j = k，则$x_{ijk}= -1$(这是因为$\theta$的系数为负数), 否则为0。$x_{ijk}$的回归系数为$\theta_k$。

### 案例

想象一下有10题目, 120个学生作答，编码为1（正确）和0（不正确）。我们想知道10个数学问题是否涉及一维尺度，以便无论受试者的能力如何，都可以根据难度对项目进行排序。

加载数据:

In [1]:
use http://www.stata.com/support/faqs/dta/raschfaq, clear

In [2]:
describe


Contains data from http://www.stata.com/support/faqs/dta/raschfaq.dta
  obs:           120                          
 vars:            11                          19 May 2005 07:47
 size:         1,440                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
math1           byte    %9.0g      Correct    math item 1
math2           byte    %9.0g      Correct    math item 2
math3           byte    %9.0g      Correc

为了对数据有一个大概的了解, 我们描述一下数据:

In [3]:
summarize math*


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       math1 |        120    .8083333    .3952626          0          1
       math2 |        120        .775    .4193332          0          1
       math3 |        120    .6833333    .4671266          0          1
       math4 |        120         .55    .4995797          0          1
       math5 |        120    .5333333    .5009794          0          1
-------------+---------------------------------------------------------
       math6 |        120         .45    .4995797          0          1
       math7 |        120        .425    .4964157          0          1
       math8 |        120          .3     .460179          0          1
       math9 |        120    .2583333    .4395535          0          1
      math10 |        120    .1666667    .3742406          0          1


为了能够拟合我们的模型, 首先需要reshape我们的数据, 将宽数据转换为长数据, 我们先看下现在的格式(宽格式):

In [4]:
list in 1/6


     +-----------------------------------------------------------------------------------------------------------------------------+
     |   math1     math2       math3       math4       math5       math6       math7       math8       math9      math10   subj_id |
     |-----------------------------------------------------------------------------------------------------------------------------|
  1. | correct   correct     correct   incorrect   incorrect   incorrect   incorrect   incorrect   incorrect   incorrect         1 |
  2. | correct   correct   incorrect   incorrect     correct     correct     correct   incorrect   incorrect   incorrect         2 |
  3. | correct   correct   incorrect   incorrect     correct   incorrect   incorrect   incorrect   incorrect   incorrect         3 |
  4. | correct   correct     correct   incorrect     correct   incorrect     correct   incorrect   incorrect   incorrect         4 |
  5. | correct   correct     correct   incorrect   incorrect     cor

转换成长格式:

In [5]:
reshape long math, i(subj_id) j(item)

(note: j = 1 2 3 4 5 6 7 8 9 10)

Data                               wide   ->   long
-----------------------------------------------------------------------------
Number of obs.                      120   ->    1200
Number of variables                  11   ->       3
j variable (10 values)                    ->   item
xij variables:
                 math1 math2 ... math10   ->   math
-----------------------------------------------------------------------------


In [7]:
list in 1/30


     +----------------------------+
     | subj_id   item        math |
     |----------------------------|
  1. |       1      1     correct |
  2. |       1      2     correct |
  3. |       1      3     correct |
  4. |       1      4   incorrect |
  5. |       1      5   incorrect |
     |----------------------------|
  6. |       1      6   incorrect |
  7. |       1      7   incorrect |
  8. |       1      8   incorrect |
  9. |       1      9   incorrect |
 10. |       1     10   incorrect |
     |----------------------------|
 11. |       2      1     correct |
 12. |       2      2     correct |
 13. |       2      3   incorrect |
 14. |       2      4   incorrect |
 15. |       2      5     correct |
     |----------------------------|
 16. |       2      6     correct |
 17. |       2      7     correct |
 18. |       2      8   incorrect |
 19. |       2      9   incorrect |
 20. |       2     10   incorrect |
     |----------------------------|
 21. |       3      1     c

按照上面的假设, 我们生成自变量$x_{ijk}$, 这k个变量的系数就是($\theta_j$), 因为有10个题目, 所以应该有10个自变量:

In [8]:
forvalues num =1/10{
      gen Th`num' = -(item==`num')
   }

看一下现在的数据, 我们得到了Th1-Th10这10个变量, 他们都是自变量, 用于预测math得分。

In [9]:
list in 1/30


     +-----------------------------------------------------------------------------------------+
     | subj_id   item        math   Th1   Th2   Th3   Th4   Th5   Th6   Th7   Th8   Th9   Th10 |
     |-----------------------------------------------------------------------------------------|
  1. |       1      1     correct    -1     0     0     0     0     0     0     0     0      0 |
  2. |       1      2     correct     0    -1     0     0     0     0     0     0     0      0 |
  3. |       1      3     correct     0     0    -1     0     0     0     0     0     0      0 |
  4. |       1      4   incorrect     0     0     0    -1     0     0     0     0     0      0 |
  5. |       1      5   incorrect     0     0     0     0    -1     0     0     0     0      0 |
     |-----------------------------------------------------------------------------------------|
  6. |       1      6   incorrect     0     0     0     0     0    -1     0     0     0      0 |
  7. |       1      7   incor

In [10]:
summarize math


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        math |      1,200        .495    .5001835          0          1


下面就可以进行固定效应条件逻辑回归, `group(subj_id)`用于表示相同的subj_id的数据来自于同一个个体。

In [14]:
clogit math Th2-Th10, group(subj_id)

note: multiple positive outcomes within groups encountered.
note: 5 groups (50 obs) dropped because of all positive or
      all negative outcomes.

Iteration 0:   log likelihood = -436.11778  
Iteration 1:   log likelihood =   -435.352  
Iteration 2:   log likelihood = -435.35069  
Iteration 3:   log likelihood = -435.35069  

Conditional (fixed-effects) logistic regression

                                                Number of obs     =      1,150
                                                LR chi2(9)        =     243.80
                                                Prob > chi2       =     0.0000
Log likelihood = -435.35069                     Pseudo R2         =     0.2188

------------------------------------------------------------------------------
        math |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         Th2 |    .241266   .3481052     0.69   0.488    -.4410077 

上面的结果中, 估计得到的系数就是我们需要的$\theta_j$, 也就是题目的难度系数，那么为什么没有Th1呢? 大家思考一下为啥没有Th1。